In [1]:
import numpy as np
import scipy.stats as ss
from decimal import Decimal, getcontext, ROUND_HALF_UP

In [2]:
codes_raw = np.loadtxt('puzzle3_input.txt', dtype=str).reshape((-1, 1))

print(codes_raw[0:10]) # View a sample

[['010000010010']
 ['011001011100']
 ['110011011101']
 ['110100100001']
 ['000110011110']
 ['110101000101']
 ['100110111001']
 ['110000000000']
 ['100101100111']
 ['001001011000']]


## Part 1

In [3]:
# Split strings into lists of chars, convert to boolean (binary)
# This bit feels like cheating, since apply_along_axis()/map() is the closest thing you can get to a for loop without explicitly writing a for loop
codes = np.apply_along_axis(lambda x: list(x[0]), axis=1, arr=codes_raw).astype(bool)

# Find most common (mode) of each column
gamma_bin = ss.stats.mode(codes)[0][0]

# Epsilon is the invert of gamma (least common bit in each column)
epsilon_bin = np.bitwise_not(gamma_bin)

# Create 16 bit representations (currently 12 bit) with zeros padded in front
gamma_b16 = np.zeros((16,), dtype=bool)
epsilon_b16 = np.zeros_like(gamma_b16, dtype=bool)
gamma_b16[4:] = gamma_bin
epsilon_b16[4:] = epsilon_bin

# Reshape, swap bytes and then view cast
gamma = np.packbits(gamma_b16.reshape(-1, 2, 8)[:, ::-1]).view(np.uint16)
epsilon = np.packbits(epsilon_b16.reshape(-1, 2, 8)[:, ::-1]).view(np.uint16)

# Convert to floats before multiplying (since they are currently unsigned 16 bit integers)
power = gamma.astype(float)*epsilon.astype(float)

In [4]:
print(f"Gamma rate = {gamma}")
print(f"Epsilon rate = {epsilon}")
print(f"Power = {power}")

Gamma rate = [1503]
Epsilon rate = [2592]
Power = [3895776.]


## Part 2

In [5]:
# Probably the most convoluted function ever, but hey... It works...
def find_rating(x, i=0, to_find=1):
    # If only one element remains, return it
    if x.shape[0] == 1:
        return x[0]
    
    # Calculate the mode and the counts
    (mode, count) = ss.stats.mode(x)
    
    # Check if the counts of the values are equal (i.e. count of the modal value = half the array length)
    # Scipy returns the lowest of the 2 modal values if this is the case, so we need to return the correct one
    # I.e. 1 for Oxygen and 0 for CO2
    if count[0, i] == x.shape[0]/2:
        m = to_find
    else:
        # If we're looking for Oxygen, then return the mode
        if to_find:
            m = mode[0, i]
        # If not, then return the opposite of the mode
        else:
            m = not mode[0, i]

    # Yay recursion!
    return find_rating(x[x[:, i] == m], i=i+1, to_find=to_find)

In [6]:
oxygen_rating_bin = find_rating(codes, i=0, to_find=1)
co2_rating_bin = find_rating(codes, i=0, to_find=0)
print(oxygen_rating_bin.astype(int))
print(co2_rating_bin.astype(int))


[0 1 1 1 1 1 0 0 0 1 1 1]
[1 1 1 1 1 0 0 0 1 1 1 0]


In [7]:
# Again, create 16 bit representations (currently 12 bit) with zeros padded in front
oxygen_b16 = np.zeros((16,), dtype=bool)
co2_b16 = np.zeros_like(gamma_b16, dtype=bool)
oxygen_b16[4:] = oxygen_rating_bin
co2_b16[4:] = co2_rating_bin

# Reshape, swap bytes and then view cast
oxygen_rating = np.packbits(oxygen_b16.reshape(-1, 2, 8)[:, ::-1]).view(np.uint16)
co2_rating = np.packbits(co2_b16.reshape(-1, 2, 8)[:, ::-1]).view(np.uint16)

print(f"Oxygen rating = {oxygen_rating}")
print(f"CO2 rating = {co2_rating}")
print(f"Life Support = {float(oxygen_rating) * float(co2_rating)}")


Oxygen rating = [1991]
CO2 rating = [3982]
Life Support = 7928162.0
